In [1]:
from train_svm import load_ml_data

In [2]:
ws = [90, 90]
winL, winR = ws

do_preprocess=True
reduced_DS=False
maxRR=True
use_RR=True
norm_RR=True
compute_morph=['wvlt', 'HOS', 'lbp', 'OurMorph']

In [3]:
tr_features, tr_labels, eval_features, eval_labels = load_ml_data(
    (winL, winR),
    do_preprocess,
    reduced_DS,
    maxRR,
    use_RR,
    norm_RR,
    compute_morph,
    cross_patient=False,
    verbose=True)

  0%|          | 0/44 [00:00<?, ?it/s]

Loading MIT BIH arr (DS12) ...


100%|██████████| 44/44 [08:10<00:00, 11.14s/it]


Saving signal processed data ...
getting rr features ...

working on rr ...
time for rr: 0.00 sec


working on norm_rr ...
time for norm_rr: 0.01 sec


working on lbp ...
(100693, 24)
time for lbp: 133.88 sec


working on wvlt ...
(100693, 47)
time for wvlt: 94.33 sec


working on HOS ...
(100693, 57)
time for HOS: 158.72 sec


working on OurMorph ...
(100693, 61)
time for OurMorph: 10.41 sec

writing pickle: /home/congyu/dataset/ECG/mitdb/ml_learning/features/w_90_90_DS12_rm_bsline_maxRR_RR_norm_RR_wvlt_HOS_lbp_OurMorph_MLII.pkl...
normalizing the data ... 


In [4]:
print(tr_features.shape)
print(tr_labels.shape)
print(eval_features.shape)
print(eval_labels.shape)

(80554, 61)
(80554,)
(20139, 61)
(20139,)


In [5]:
from utils import calc_class_weights
class_weights = calc_class_weights(tr_labels)

In [6]:
from sklearn import svm

In [7]:
svm_model = svm.SVC(C=0.1, 
                    kernel='rbf', 
                    degree=3, 
                    gamma='auto',
                    coef0=0.0, 
                    shrinking=True, 
                    probability=False, 
                    tol=0.001,
                    cache_size=200, 
                    class_weight=class_weights, 
                    max_iter=-1, 
                    decision_function_shape="ovo", 
                    random_state=None)

In [8]:
from utils import PrintTime

In [9]:
with PrintTime("training the model"):
    svm_model.fit(tr_features, tr_labels)


working on training the model ...
time for training the model: 182.94 sec



In [11]:
from sklearn.metrics import f1_score, classification_report
from config import AAMI_CLASSES

In [13]:
y_pred = svm_model.predict(eval_features)

In [14]:
print("macro f1 score", f1_score(eval_labels, y_pred, average='macro'))

marco f1 score 0.8256143183983469


In [15]:
print(classification_report(eval_labels, y_pred, target_names=AAMI_CLASSES))

              precision    recall  f1-score   support

           F       0.43      0.90      0.58       176
           N       1.00      0.97      0.98     18019
        SVEB       0.67      0.95      0.79       577
         VEB       0.93      0.99      0.96      1367

    accuracy                           0.97     20139
   macro avg       0.75      0.95      0.83     20139
weighted avg       0.98      0.97      0.97     20139



In [16]:
model_svm_path="/home/congyu/dataset/ECG/mitdb/ml_learning/svm_models/baseline_model.joblib.pkl"

In [ ]:
import os
import joblib

In [ ]:
if model_svm_path:
    # Export model: save/write trained SVM model
    if not os.path.exists(os.path.dirname(model_svm_path)):
        os.makedirs(os.path.dirname(model_svm_path))
    joblib.dump(svm_model, model_svm_path)

In [ ]:
decision_ovo = svm_model.decision_function(eval_features)

In [ ]:
predict_ovo, counter = ovo_voting(decision_ovo, 4)

In [ ]:
perf_measures = compute_AAMI_performance_measures(predict_ovo, labels)
write_AAAMI_results_gamma(output_path, gamma_value, perf_measures, C_value, voting_strategy, DS)